In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

# 해설
# memory = ConversationBufferMemory(
#     llm=llm,
#     return_messages=True,
# )
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=5,
)

# memory = ConversationSummaryBufferMemory(
#     llm=llm,
#     max_token_limit=120,
#     return_messages=True,
# )


examples = [
    {
        "movie": "명량",
        "information": """
영화: 명량
이모티콘: 🚢👑👏
설명: 🚢 배 - 명량 해전을 배경으로 한 역사적인 전쟁 영화
     👑 왕관 - 조선시대 조선왕의 힘과 권력을 상징
     👏 박수 - 관객들로부터 받은 호평과 찬사를 나타냄
        """,
    },
    {
        "movie": "아바타",
        "information": """
영화: 아바타
이모티콘: 🌿🌌👽
설명: 🌿 자연 - 영화 속 판도라 행성의 아름다운 자연환경
     🌌 우주 - 판도라 행성의 신비로운 우주적 배경
     👽 외계인 - 판도라 행성에 사는 나비족과의 만남과 소통을 다룬 과학-fantasy 영화            
        """,
    },
    {
        "movie": "타이타닉",
        "information": """
영화: 타이타닉
이모티콘: 🚢💔🎶
설명: 🚢 배 - 타이타닉호를 상징
     💔 심장 - 잭과 로즈의 슬픈 사랑 이야기
     🎶 음악 - 영화의 감동적인 음악과 사운드트랙
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "영화 {movie}에 대해서 알려줘"),
        ("ai", "{information}"),
    ]
)

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 해설 내용
# final_prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a movie expert. You know every movie. If a human tells you the title of the movie, you have to respond with 3 emoticons.",
#         ),
#         fewshot_prompt,
#         (
#             "system",
#             "The above examples should not be provided to the user. The user can only be provided with the conversation record below. Please provide the information to the user using the record below.",
#         ),
#         MessagesPlaceholder(variable_name="history"),
#         ("human", "{question}"),
#     ]
# )
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화 전문가로서 모든 영화에 대해잘 알고 있어. 영화를 이모티콘으로 표현하는 게임을 해보자."),
        MessagesPlaceholder(variable_name="history"),
        fewshot_prompt,
        ("human", "영화 {movie}에 대해서 알려줘"),
    ]
)


def get_history():
    return memory.load_memory_variables({})

def load_memory(_):
    return memory.load_memory_variables({})["history"]


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result.content)
    

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm
invoke_chain("추격자")


영화: 추격자
이모티콘: 🏃‍♂️🔫🚗
설명: 🏃‍♂️ 주인공 - 무법자를 추격하는 경찰
     🔫 총 - 격렬한 액션과 총격전을 상징
     🚗 차 - 카체이스와 스릴 넘치는 추격씬이 인상적인 영화


In [2]:

invoke_chain("300")


영화: 300
이모티콘: 🛡️💪🏛️
설명: 🛡️ 스파르타 군인들 - 300명의 용사들이 전투에 나섬
     💪 전투 - 스파르타 군인들의 강인한 신체력과 전투기술을 보여줌
     🏛️ 테르모필레의 전투 - 역사적인 테르모필레 전투를 다룬 영화


In [3]:
invoke_chain("맨인블랙")

영화: 맨 인 블랙
이모티콘: 👽🕶️🔫
설명: 👽 외계인 - 지구를 지키는 비밀 기관 맨 인 블랙이 다양한 외계인들과 상호작용
     🕶️ 세련된 스타일 - 맨 인 블랙이 입는 검은 슈트와 선글라스로 유명
     🔫 무장 - 맨 인 블랙이 사용하는 고급 장비와 무장한 모습


In [5]:
invoke_chain("처음에 질문한 영화에 대해서 다시 알려줘")

죄송합니다. 처음에 질문한 영화가 "추격자" 였습니다.

영화: 추격자
이모티콘: 🏃‍♂️🔫🚗
설명: 🏃‍♂️ 주인공 - 무법자를 추격하는 경찰
     🔫 총 - 격렬한 액션과 총격전을 상징
     🚗 차 - 카체이스와 스릴 넘치는 추격씬이 인상적인 영화

더 궁금한 점이 있으시면 언제든지 물어보세요!


In [6]:
invoke_chain("두번째 질문한 영화에 대해서 다시 알려줘")

두번째 질문한 영화가 "300" 이었습니다.

영화: 300
이모티콘: 🛡️💪🏛️
설명: 🛡️ 스파르타 군인들 - 300명의 용사들이 전투에 나섬
     💪 전투 - 스파르타 군인들의 강인한 신체력과 전투기술을 보여줌
     🏛️ 테르모필레의 전투 - 역사적인 테르모필레 전투를 다룬 영화

더 궁금한 점이 있으시면 언제든지 물어보세요!


In [7]:
get_history()

{'history': [HumanMessage(content='추격자'),
  AIMessage(content='영화: 추격자\n이모티콘: 🏃\u200d♂️🔫🚗\n설명: 🏃\u200d♂️ 주인공 - 무법자를 추격하는 경찰\n     🔫 총 - 격렬한 액션과 총격전을 상징\n     🚗 차 - 카체이스와 스릴 넘치는 추격씬이 인상적인 영화'),
  HumanMessage(content='300'),
  AIMessage(content='영화: 300\n이모티콘: 🛡️💪🏛️\n설명: 🛡️ 스파르타 군인들 - 300명의 용사들이 전투에 나섬\n     💪 전투 - 스파르타 군인들의 강인한 신체력과 전투기술을 보여줌\n     🏛️ 테르모필레의 전투 - 역사적인 테르모필레 전투를 다룬 영화'),
  HumanMessage(content='맨인블랙'),
  AIMessage(content='영화: 맨 인 블랙\n이모티콘: 👽🕶️🔫\n설명: 👽 외계인 - 지구를 지키는 비밀 기관 맨 인 블랙이 다양한 외계인들과 상호작용\n     🕶️ 세련된 스타일 - 맨 인 블랙이 입는 검은 슈트와 선글라스로 유명\n     🔫 무장 - 맨 인 블랙이 사용하는 고급 장비와 무장한 모습'),
  HumanMessage(content='처음에 질문한 영화에 대해서 다시 알려줘'),
  AIMessage(content='죄송합니다. 처음에 질문한 영화가 "추격자" 였습니다.\n\n영화: 추격자\n이모티콘: 🏃\u200d♂️🔫🚗\n설명: 🏃\u200d♂️ 주인공 - 무법자를 추격하는 경찰\n     🔫 총 - 격렬한 액션과 총격전을 상징\n     🚗 차 - 카체이스와 스릴 넘치는 추격씬이 인상적인 영화\n\n더 궁금한 점이 있으시면 언제든지 물어보세요!'),
  HumanMessage(content='두번째 질문한 영화에 대해서 다시 알려줘'),
  AIMessage(content='두번째 질문한 영화가 "300" 이었습니다.\n\n영화: 300\n이모티콘: 🛡️💪🏛️\n설명: 🛡️ 스파르